# **LifeHack 2020- Problem #2**

Vision Impairment can be a serious implication on one’s health. According to the key facts stated by the World Health Organization “Globally, at least 2.2 billion people have a vision impairment or blindness, of whom at least 1 billion have a vision impairment that could have been prevented or has yet to be addressed.” Do you have the ability to better detect potential vision issues and help change thousands, if not, millions of lives? 

For example, the given dataset looks at cataract and normal eye image dataset for cataract detection. Participants are encouraged to use other available datasets to better analyse and share their findings regarding the detection of the potential risk of visual problems detection, preferably relating to the Asia region. Indeed this problem statement covers a wide area of visual impairments and does not need to focus solely on cataract. Participants are free to explore other options of eye disease detection.  

  • Present Insight into the data by mean of visualization and/or interactive means
  • Prediction model for eye disease 

For reference:
https://www.kaggle.com/jr2ngb/cataractdataset?Vision 

In [3]:
import pandas as pd                # Python Panel DataFrames
import matplotlib                  # 2D Plotting Library
import matplotlib.pyplot as plt
import seaborn as sns              # Python Data Visualization Library based on matplotlib
#import geopandas as gpd            # Python Geospatial Data Library
plt.style.use('fivethirtyeight')
%matplotlib inline

ModuleNotFoundError: No module named 'seaborn'